# IMPORTAÇÃO DE BIBLIOTECAS

In [254]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [255]:
# COMMON VARIABLES

alg_to_analyze = [
    'vader_sentiment_analysis_score',
    'vader_sentiment_analysis_polarity'
]

tpd_columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

# LEITURA DE ARQUIVOS

In [256]:
!python3 '/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/main_generate_json_files.py' --all_tweets True

Starting script...
Removing existing file...
Ending script...


In [257]:
data = pd.read_json(r'/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/alldata.json')
data.head()

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
id_str_twitter,23781146,15148309,927028057763995648,77648005,3429765093,10794662,1034514325644431361,529445600,12050562,62314491,18289937,172880784,18609547,2164986296,1026783544457273345,109604376,1040939562774790144,69734383,119897261
participant_id,1,2,3,None,4,5,6,None,7,8,None,9,10,11,12,13,14,15,16
name,Fe Rebelatto,Sommelier de Airbnb,✪ Felippe,Conference Distinguished Specialist,Paula Santana,Maharaja of the Legacies,Jessica,"var nickname = ""Riquettinha"";",Roberta Arcoverde,Gabriela,Thamara Andrade,felipe.ts ⚡,Juan Lopes,Rafael Dantas,iagor com i,T̷o̷m̷o̷y̷o̷ ̷ F̷u̷r̷a̷c̷ã̷o̷,Esdras Xavier,belle-🏴.dev || twitch.tv/bellesamways,Willian Justen
screen_name,rebelatto,Guilh_rm_,FelippeRegazio,NannoKa,psanrosa13,rponte,dev_jessi,Riquettinha,rla4,Gabrielathalita,thamyk,felipefialho_,juanplopes,RaffaelDantass,Iagor51,RafaelMansilha,esdras_xavieer,bellesamways,Willian_justen
location,"Rio de Janeiro, Brasil",somewhere in Brazil,,Ctba-PR Brasil,Brasil,"Ceará, Brazil",São Paulo - Brasil,"Belo Horizonte, Brasil","Sao Paulo, Brazil",Belo Horizonte,"Belo Horizonte, Brazil.",Dark Side of the Moon,"Rio de Janeiro, Brasil","São Paulo, Brasil",Pvh,"Palmas, Tocantins, Brasil","Jaraguá do Sul, Brasil",Curitiba,"Petrópolis, Brasil"


In [258]:
data_psi = pd.read_csv(r'/Users/leosilva/Documents/Estudo/Doutorado/Coimbra/2019-2020/Disciplinas/Thesis/analises_psicologos/Tweets_consolidados.csv')
data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3
0,102600,Negativo Fraco,Positivo Fraco,Neutro
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte
2,103085,Negativo Forte,Negativo Forte,Negativo Forte
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte


In [259]:
data_psi_divergentes_resolvidos = pd.read_csv(r'/Users/leosilva/Documents/Estudo/Doutorado/Coimbra/2019-2020/Disciplinas/Thesis/analises_psicologos/Tweets_divergentes_resultado.csv')
data_psi_divergentes_resolvidos.head()

,ID,Classificacao
0,102600,Neutro
1,103241,Negativo Fraco
2,103875,Positivo Fraco
3,104229,Neutro
4,104337,Negativo Fraco


In [260]:
data_participants = pd.read_csv(r'/Users/leosilva/Documents/Estudo/Doutorado/Coimbra/2019-2020/Disciplinas/Thesis/tweets_dev/classification/tweets_consolidados.csv')
print(len(data_participants))
data_participants.head()
data_participants['Id_usuario'].value_counts()

455


26    35
28    35
30    35
31    35
34    35
35    35
37    35
38    35
39    35
40    35
41    35
42    35
43    35
Name: Id_usuario, dtype: int64

## Preparação do DataFrame das respostas das psicólogas

In [261]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI1'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI1'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI1'] == 'Neutro'),
    (data_psi['Classificacao_PSI1'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI1'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI1_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos


In [262]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI2'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI2'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI2'] == 'Neutro'),
    (data_psi['Classificacao_PSI2'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI2'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI2_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg


In [263]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI3'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI3'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI3'] == 'Neutro'),
    (data_psi['Classificacao_PSI3'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI3'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI3_3_cat'] = np.select(conditions, values)

data_psi.columns = ['id',
                    'Classificacao_PSI1',
                    'Classificacao_PSI2',
                    'Classificacao_PSI3',
                    'Classificacao_PSI1_3_cat',
                    'Classificacao_PSI2_3_cat',
                    'Classificacao_PSI3_3_cat']

data_psi.head()

,id,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat,Classificacao_PSI3_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos,neu
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg,neg


In [264]:
# create a list of our conditions
conditions = [
    (data_psi_divergentes_resolvidos['Classificacao'] == 'Positivo Fraco'),
    (data_psi_divergentes_resolvidos['Classificacao'] == 'Positivo Forte'),
    (data_psi_divergentes_resolvidos['Classificacao'] == 'Neutro'),
    (data_psi_divergentes_resolvidos['Classificacao'] == 'Negativo Fraco'),
    (data_psi_divergentes_resolvidos['Classificacao'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi_divergentes_resolvidos['Classificacao'] = np.select(conditions, values)

data_psi_divergentes_resolvidos.head()

,ID,Classificacao
0,102600,neu
1,103241,neg
2,103875,pos
3,104229,neu
4,104337,neg


In [265]:
# data_psi.columns = ['id',
#                     'Class_PSI1_5_cat',
#                     'Class_PSI2_5_cat',
#                     'Class_PSI3_5_cat',
#                     'Class_PSI1_3_cat',
#                     'Class_PSI2_3_cat',
#                     'Class_PSI3_3_cat']

# data_psi.head()

In [266]:
# # create a list of our conditions
# conditions = [
#     (data_psi['Class_PSI1_5_cat'] == 'Positivo Fraco'),
#     (data_psi['Class_PSI1_5_cat'] == 'Positivo Forte'),
#     (data_psi['Class_PSI1_5_cat'] == 'Neutro'),
#     (data_psi['Class_PSI1_5_cat'] == 'Negativo Fraco'),
#     (data_psi['Class_PSI1_5_cat'] == 'Negativo Forte')
#     ]

# # create a list of the values we want to assign for each condition
# values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# # create a new column and use np.select to assign values to it using our lists as arguments
# data_psi['Class_PSI1_5_cat'] = np.select(conditions, values)

# data_psi.head()

In [267]:
# # create a list of our conditions
# conditions = [
#     (data_psi['Class_PSI2_5_cat'] == 'Positivo Fraco'),
#     (data_psi['Class_PSI2_5_cat'] == 'Positivo Forte'),
#     (data_psi['Class_PSI2_5_cat'] == 'Neutro'),
#     (data_psi['Class_PSI2_5_cat'] == 'Negativo Fraco'),
#     (data_psi['Class_PSI2_5_cat'] == 'Negativo Forte')
#     ]

# # create a list of the values we want to assign for each condition
# values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# # create a new column and use np.select to assign values to it using our lists as arguments
# data_psi['Class_PSI2_5_cat'] = np.select(conditions, values)

# data_psi.head()

In [268]:
# # create a list of our conditions
# conditions = [
#     (data_psi['Class_PSI3_5_cat'] == 'Positivo Fraco'),
#     (data_psi['Class_PSI3_5_cat'] == 'Positivo Forte'),
#     (data_psi['Class_PSI3_5_cat'] == 'Neutro'),
#     (data_psi['Class_PSI3_5_cat'] == 'Negativo Fraco'),
#     (data_psi['Class_PSI3_5_cat'] == 'Negativo Forte')
#     ]

# # create a list of the values we want to assign for each condition
# values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# # create a new column and use np.select to assign values to it using our lists as arguments
# data_psi['Class_PSI3_5_cat'] = np.select(conditions, values)

# data_psi.head()

In [269]:
print(len(data_psi))

455


## Preparação do DataFrame das respostas dos participantes

In [270]:
# create a list of our conditions
conditions = [
    (data_participants['classificacao'] == 'Positivo Fraco'),
    (data_participants['classificacao'] == 'Positivo Forte'),
    (data_participants['classificacao'] == 'Neutro'),
    (data_participants['classificacao'] == 'Negativo Fraco'),
    (data_participants['classificacao'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_participants['classificacao_3_cat'] = np.select(conditions, values)

print(len(data_participants))
data_participants.head()

455


,id,classificacao,Id_usuario,classificacao_3_cat
0,197052,Positivo Fraco,43,pos
1,197097,Neutro,43,neu
2,198955,Neutro,43,neu
3,200154,Positivo Fraco,43,pos
4,200645,Positivo Fraco,43,pos


In [271]:
# # create a list of our conditions
# conditions = [
#     (data_participants['classificacao'] == 'Positivo Fraco'),
#     (data_participants['classificacao'] == 'Positivo Forte'),
#     (data_participants['classificacao'] == 'Neutro'),
#     (data_participants['classificacao'] == 'Negativo Fraco'),
#     (data_participants['classificacao'] == 'Negativo Forte')
#     ]

# # create a list of the values we want to assign for each condition
# values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# # create a new column and use np.select to assign values to it using our lists as arguments
# data_participants['classificacao_5_cat'] = np.select(conditions, values)

# data_participants.head()

In [272]:
# print(data_psi['Class_PSI3_5_cat'].value_counts())

In [273]:
tweets = []

for d in data:
    if data[d]['bigfive']:
        tweets.append(data[d]['tweets'])

final_tweets = []
for i in tweets:
    final_tweets.extend(i)

In [274]:
print(len(final_tweets))

91632


In [275]:
def fill_manual_polarities_3_cat(df):
    for d in df['id']:
        pol = []
        t = df.query('id == {}'.format(d))
        pol.append(t['Classificacao_PSI1_3_cat'].values[0])
        pol.append(t['Classificacao_PSI2_3_cat'].values[0])
        pol.append(t['Classificacao_PSI3_3_cat'].values[0])
        c = dict(Counter(pol))
        for i in c.items():
            if i[1] >= 2:
                df.loc[df['id'] == d, 'manual_polarity_3_cat'] = i[0]
    
    return df

In [276]:
# def fill_manual_polarities_5_cat(df):
#     for d in df['id']:
#         pol = []
#         t = df.query('id == {}'.format(d))
#         pol.append(t['Class_PSI1_5_cat'].values[0])
#         pol.append(t['Class_PSI2_5_cat'].values[0])
#         pol.append(t['Class_PSI3_5_cat'].values[0])
#         c = dict(Counter(pol))
#         for i in c.items():
#             if i[1] >= 2:
#                 df.loc[df['id'] == d, 'manual_polarity_5_cat'] = i[0]
#     return df

In [277]:
def fill_nan_values_by_consolidated_polarities(df, df_consolidated):
    temp_df = df[df['manual_polarity_3_cat'].isna()]
    for i in temp_df['id']:
        df.loc[df['id'] == i, 'manual_polarity_3_cat'] = df_consolidated[df_consolidated['ID'] == i]['Classificacao'].values[0]
    return df

In [278]:
def conf_matrix(x, col1, col2):
    if x[col1] == 1 and x[col2] == 1:
        return 'TP'
    elif x[col1] == 1 and x[col2] == -1:
        return 'FN'
    elif x[col1] == -1 and x[col2] == 1:
        return 'FP'
    elif x[col1] == -1 and x[col2] == -1:
        return 'TN'
    else:
        return 0

In [279]:
rdf = pd.DataFrame(columns=['Sentiment Lexicon', 'Accuracy', 'Precision - Positive', 
                                'Recall - Positive', 'F1-Score - Positive',
                                'Precision - Negative', 
                                'Recall - Negative', 'F1-Score - Negative'
                               ])

In [280]:
data_psi.head()

,id,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat,Classificacao_PSI3_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos,neu
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg,neg


In [281]:
print(len(data_psi))
print(data_psi['Classificacao_PSI1_3_cat'].value_counts())
print(data_psi['Classificacao_PSI2_3_cat'].value_counts())
print(data_psi['Classificacao_PSI3_3_cat'].value_counts())

455
pos    245
neg    107
neu    103
Name: Classificacao_PSI1_3_cat, dtype: int64
pos    221
neg    175
neu     59
Name: Classificacao_PSI2_3_cat, dtype: int64
neu    306
pos     80
neg     69
Name: Classificacao_PSI3_3_cat, dtype: int64


In [282]:
df_data_psi_3_cat = fill_manual_polarities_3_cat(data_psi)
print(len(df_data_psi_3_cat))
df_data_psi_3_cat = fill_nan_values_by_consolidated_polarities(df_data_psi_3_cat, data_psi_divergentes_resolvidos)
print(len(df_data_psi_3_cat))
# df_data_psi_3_cat = df_data_psi_3_cat.dropna()
df_data_psi_3_cat.head()

455
455


,id,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat,Classificacao_PSI3_3_cat,manual_polarity_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos,neu,neu
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg,neg,neg


In [283]:
# df_data_psi_5_cat = fill_manual_polarities_5_cat(data_psi)
# df_data_psi_5_cat.head()

In [284]:
# df_data_psi_5_cat = df_data_psi_5_cat.dropna()
# df_data_psi_5_cat.head()

In [285]:
# print(len(df_data_psi_5_cat))

In [286]:
def create_psycho_versus_participants_df(df, category):

    if category == 3:
        fields = {
            'd': ['manual_polarity_3_cat', 'participants_polarity_3_cat'],
            'r': ['manual_polarity_3_cat', 'classificacao_3_cat']
        }
    elif category == 5:
        fields = {
            'd': ['manual_polarity_5_cat', 'participants_polarity_5_cat'],
            'r': ['manual_polarity_5_cat', 'classificacao_5_cat']
        }


    d = {
        'id':[],
        fields['d'][0]:[],
        fields['d'][1]:[]
    }

    ids_not_found = []

    count = 0
    for idx in data_participants['id']:
        found = False
        for i in df['id']:
            if idx == i:
                found = True
                count = count + 1
                t = df.query('id == {}'.format(i))
                d['id'].append(t[t.keys()[0]].values[0])
                d[fields['d'][0]].append(t[fields['r'][0]].tolist()[0])
                p_t = data_participants.loc[data_participants['id'] == idx]
                d[fields['d'][1]].append(p_t[fields['r'][1]].tolist()[0])
        if found == False:
            ids_not_found.append(idx)

    print("ids not found: ", len(ids_not_found))
    temp_pd = pd.DataFrame(d)
    return temp_pd

In [287]:
def perform_statistics_between_psycho_participants(df, cat, prefix):
#     if cat == 3:
    df['participants_prediction'] = df['participants_polarity_3_cat'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
    df['psychologists_prediction'] = df['manual_polarity_3_cat'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
#     elif cat == 5:
#         if prefix == 'wp':
#             df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 'w_neg' or x == 's_neg' else (1 if x == 'w_pos' else 0))
#             df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 'w_neg' or x == 's_neg' else (1 if x == 'w_pos' else 0))
#         elif prefix == 'sp':
#             df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 's_neg' or x == 'w_neg' else (1 if x == 's_pos' else 0))
#             df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 's_neg' or x == 'w_neg' else (1 if x == 's_pos' else 0))
#         elif prefix == 'wn':
#             df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 'w_pos' or x == 's_pos' else (1 if x == 'w_neg' else 0))
#             df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 'w_pos' or x == 's_pos' else (1 if x == 'w_neg' else 0))
#         elif prefix == 'sn':
#             df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 's_pos' or x == 'w_pos' else (1 if x == 's_neg' else 0))
#             df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 's_pos' or x == 'w_pos' else (1 if x == 's_neg' else 0))
    
    df['accuracy'] = df.apply(lambda x: 1 if x['participants_prediction'] == x['psychologists_prediction'] else 0, axis=1)

    df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'psychologists_prediction', 'participants_prediction'), axis=1)

    list_precision = []

    conf_vals = df['conf_matrix'].value_counts().to_dict()
    
    if 'FP' not in conf_vals:
        conf_vals['FP'] = 0
    if 'TP' not in conf_vals:
        conf_vals['TP'] = 0
    if 'FN' not in conf_vals:
        conf_vals['FN'] = 0    
    if 'TN' not in conf_vals:
        conf_vals['TN'] = 0
        
    print(conf_vals)

    accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
    precision_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])

    # ------------------------------------------------------
    # verificar se devo incluir os NEUTROS também nas contas
    # ------------------------------------------------------

    recall_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
    f1_score_pos = 2*(precision_pos*recall_pos / (precision_pos + recall_pos))

    precision_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FN'])
    recall_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FP'])
    f1_score_neg = 2*(precision_neg*recall_neg / (precision_neg + recall_neg))

    d = {
        'Accuracy': accuracy,
        'Precision - Positive': precision_pos,
        'Recall - Positive': recall_pos,
        'F1-Score - Positive': f1_score_pos,
        'Precision - Negative': precision_neg,
        'Recall - Negative': recall_neg,
        'F1-Score - Negative': f1_score_neg,
        'F1-Score - Average': (f1_score_pos + f1_score_neg) / 2
    }
       
    return pd.DataFrame.from_records([d])

In [288]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_3_cat, 3)
print(len(df_for_analysis))
pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 3, '')
pd_stats_psy_part

ids not found:  35
420
{0: 175, 'TP': 137, 'TN': 74, 'FP': 23, 'FN': 11}


,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative,F1-Score - Average
0,0.861224,0.85625,0.925676,0.88961,0.870588,0.762887,0.813187,0.851399


In [289]:
# df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
# print(len(df_for_analysis))
# pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'wp')
# pd_stats_psy_part

In [290]:
# df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
# print(len(df_for_analysis))
# pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'sp')
# pd_stats_psy_part

In [291]:
# df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
# print(len(df_for_analysis))
# pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'wn')
# pd_stats_psy_part

In [292]:
# df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
# print(len(df_for_analysis))
# pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'sn')
# pd_stats_psy_part

In [293]:
# df_data_psi_5_cat.head()

In [294]:
df_for_analysis.head()

,id,manual_polarity_3_cat,participants_polarity_3_cat,participants_prediction,psychologists_prediction,accuracy,conf_matrix
0,179689,pos,pos,1,1,1,TP
1,179701,neu,pos,1,0,0,0
2,179748,neg,neg,-1,-1,1,TN
3,179886,pos,pos,1,1,1,TP
4,179917,neg,neg,-1,-1,1,TN


In [295]:
# df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)

# # REMOVENDO OS NEUTROS
# df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['manual_polarity_5_cat']=='neu'].index, inplace=True)
# df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['participants_polarity_5_cat']=='neu'].index, inplace=True)

# test = df_for_analysis['participants_polarity_5_cat']
# pred = df_for_analysis['manual_polarity_5_cat']
# confusion = confusion_matrix(test, pred)
# print('Confusion Matrix\n')
# print(confusion)

# print('\nAccuracy: {:.6f}\n'.format(accuracy_score(test, pred)))

# print('Micro Precision: {:.6f}'.format(precision_score(test, pred, average='micro')))
# print('Micro Recall: {:.6f}'.format(recall_score(test, pred, average='micro')))
# print('Micro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='micro')))

# print('Macro Precision: {:.6f}'.format(precision_score(test, pred, average='macro')))
# print('Macro Recall: {:.6f}'.format(recall_score(test, pred, average='macro')))
# print('Macro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='macro')))

# print('Weighted Precision: {:.6f}'.format(precision_score(test, pred, average='weighted')))
# print('Weighted Recall: {:.6f}'.format(recall_score(test, pred, average='weighted')))
# print('Weighted F1-score: {:.6f}'.format(f1_score(test, pred, average='weighted')))

# print('\nClassification Report\n')
# print(classification_report(test, pred, digits=6))

In [296]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_3_cat, 3)

print("BEFORE REMOVING NEUTRAL: ", len(df_for_analysis))

# REMOVENDO OS NEUTROS
df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['manual_polarity_3_cat']=='neu'].index, inplace=True)
df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['participants_polarity_3_cat']=='neu'].index, inplace=True)

print("AFTER REMOVING NEUTRAL: ", len(df_for_analysis))

test = df_for_analysis['participants_polarity_3_cat']
pred = df_for_analysis['manual_polarity_3_cat']
confusion = confusion_matrix(test, pred)
print('Confusion Matrix\n')
print(confusion)

print('\nAccuracy: {:.6f}\n'.format(accuracy_score(test, pred)))

print('Micro Precision: {:.6f}'.format(precision_score(test, pred, average='micro')))
print('Micro Recall: {:.6f}'.format(recall_score(test, pred, average='micro')))
print('Micro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='micro')))

print('Macro Precision: {:.6f}'.format(precision_score(test, pred, average='macro')))
print('Macro Recall: {:.6f}'.format(recall_score(test, pred, average='macro')))
print('Macro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='macro')))

print('Weighted Precision: {:.6f}'.format(precision_score(test, pred, average='weighted')))
print('Weighted Recall: {:.6f}'.format(recall_score(test, pred, average='weighted')))
print('Weighted F1-score: {:.6f}'.format(f1_score(test, pred, average='weighted')))

print('\nClassification Report\n')
print(classification_report(test, pred, digits=6))

ids not found:  35
BEFORE REMOVING NEUTRAL:  420
AFTER REMOVING NEUTRAL:  245
Confusion Matrix

[[ 74  11]
 [ 23 137]]

Accuracy: 0.861224

Micro Precision: 0.861224
Micro Recall: 0.861224
Micro F1-score: 0.861224

Macro Precision: 0.844281
Macro Recall: 0.863419
Macro F1-score: 0.851399

Weighted Precision: 0.869198
Weighted Recall: 0.861224
Weighted F1-score: 0.863096

Classification Report

              precision    recall  f1-score   support

         neg   0.762887  0.870588  0.813187        85
         pos   0.925676  0.856250  0.889610       160

    accuracy                       0.861224       245
   macro avg   0.844281  0.863419  0.851399       245
weighted avg   0.869198  0.861224  0.863096       245

